In [1]:
import numpy as np
import tensorflow as tf
import os

import sionna
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, ITURadioMaterial, PathSolver

import mitsuba as mi
import drjit as dr

%matplotlib
import matplotlib.pyplot as plt

# mi.set_variant('scalar_rgb')

2025-04-07 07:59:43.836063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744027183.852361  158354 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744027183.857306  158354 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744027183.870435  158354 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744027183.870455  158354 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744027183.870457  158354 computation_placer.cc:177] computation placer alr

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [2]:
scene_dir = "minis"
scene_files = [f for f in os.listdir(scene_dir) if f.endswith('.xml')]
print(scene_files)



['row0_col4.xml', 'row5_col1.xml', 'row7_col0.xml', 'row2_col5.xml', 'row5_col6.xml', 'row0_col1.xml', 'row3_col5.xml', 'row1_col2.xml', 'row4_col2.xml', 'row5_col7.xml', 'row7_col4.xml', 'row2_col3.xml', 'row4_col6.xml', 'row5_col0.xml', 'row0_col0.xml', 'row2_col1.xml', 'row2_col4.xml', 'row0_col6.xml', 'row0_col5.xml', 'row4_col0.xml', 'row3_col2.xml', 'row4_col4.xml', 'row7_col7.xml', 'row4_col3.xml', 'row2_col7.xml', 'row5_col2.xml', 'row5_col4.xml', 'row6_col3.xml', 'row4_col5.xml', 'row6_col2.xml', 'row3_col6.xml', 'row7_col5.xml', 'row6_col5.xml', 'row4_col1.xml', 'row7_col1.xml', 'row3_col7.xml', 'row1_col6.xml', 'row5_col3.xml', 'row6_col7.xml', 'row0_col7.xml', 'row7_col3.xml', 'row6_col0.xml', 'row3_col1.xml', 'row3_col4.xml', 'row3_col3.xml', 'row1_col3.xml', 'row6_col1.xml', 'row0_col3.xml', 'row5_col5.xml', 'row1_col4.xml', 'row6_col4.xml', 'row7_col6.xml', 'row1_col7.xml', 'row2_col2.xml', 'row3_col0.xml', 'row1_col5.xml', 'row1_col0.xml', 'row2_col0.xml', 'row4_col7.xm

In [10]:
scene_file = scene_files[0]
scene = load_scene(os.path.join(scene_dir, scene_file))

try:
    os.mkdir(os.path.splitext(scene_file)[0])
except OSError:
    print('dir already exists')

In [4]:

rx_points = []
min = (0,0, 0)
max = (500, 500, 1000)

x_start, y_start, _ = min
x_end, y_end, _ = max
xx = np.linspace(x_start, x_end, 25)
yy = np.linspace(y_start, y_end, 25)
ray_pos = np.array([(x, y, 1000) for x in xx for y in yy])

z_rays = mi.Ray3f(o=ray_pos.T, d=(0, 0, -1))
z_hits = scene.mi_scene.ray_intersect(z_rays)

for z_hit, valid in zip(z_hits.p.numpy().T, z_hits.is_valid()):
    if (not valid):
        continue
    start = int(np.ceil(z_hit[2] / 25) * 25)
    print(start)
    for i in range(start, 200, 25):
        print([z_hit[0], z_hit[1], i])
        rx_points.append(np.array([z_hit[0], z_hit[1], i]))
    # rx_points.append(z_hit)
# print(rx_points)

25
[83.33334, 312.5, 25]
[83.33334, 312.5, 50]
[83.33334, 312.5, 75]
[83.33334, 312.5, 100]
[83.33334, 312.5, 125]
[83.33334, 312.5, 150]
[83.33334, 312.5, 175]
25
[125.0, 312.5, 25]
[125.0, 312.5, 50]
[125.0, 312.5, 75]
[125.0, 312.5, 100]
[125.0, 312.5, 125]
[125.0, 312.5, 150]
[125.0, 312.5, 175]
25
[145.83333, 312.49997, 25]
[145.83333, 312.49997, 50]
[145.83333, 312.49997, 75]
[145.83333, 312.49997, 100]
[145.83333, 312.49997, 125]
[145.83333, 312.49997, 150]
[145.83333, 312.49997, 175]
25
[145.83334, 333.3333, 25]
[145.83334, 333.3333, 50]
[145.83334, 333.3333, 75]
[145.83334, 333.3333, 100]
[145.83334, 333.3333, 125]
[145.83334, 333.3333, 150]
[145.83334, 333.3333, 175]
25
[145.83333, 354.16663, 25]
[145.83333, 354.16663, 50]
[145.83333, 354.16663, 75]
[145.83333, 354.16663, 100]
[145.83333, 354.16663, 125]
[145.83333, 354.16663, 150]
[145.83333, 354.16663, 175]
25
[145.83333, 375.0, 25]
[145.83333, 375.0, 50]
[145.83333, 375.0, 75]
[145.83333, 375.0, 100]
[145.83333, 375.0, 125

In [5]:
## Include points that did not have a valid hitpoint due to lack of floor right now
for i, v in enumerate(z_hits.is_valid()):
    print(i, v)
    if(not v):
        print(ray_pos[i])
        r = ray_pos[i]
        for i in range(0, 200, 25):
            rx_points.append(np.array([r[0], r[1], i]))
        # rx_points.append(np.array([r[0], r[1], 0]))

0 False
[   0.    0. 1000.]
1 False
[   0.           20.83333333 1000.        ]
2 False
[   0.           41.66666667 1000.        ]
3 False
[   0.    62.5 1000. ]
4 False
[   0.           83.33333333 1000.        ]
5 False
[   0.          104.16666667 1000.        ]
6 False
[   0.  125. 1000.]
7 False
[   0.          145.83333333 1000.        ]
8 False
[   0.          166.66666667 1000.        ]
9 False
[   0.   187.5 1000. ]
10 False
[   0.          208.33333333 1000.        ]
11 False
[   0.          229.16666667 1000.        ]
12 False
[   0.  250. 1000.]
13 False
[   0.          270.83333333 1000.        ]
14 False
[   0.          291.66666667 1000.        ]
15 False
[   0.   312.5 1000. ]
16 False
[   0.          333.33333333 1000.        ]
17 False
[   0.          354.16666667 1000.        ]
18 False
[   0.  375. 1000.]
19 False
[   0.          395.83333333 1000.        ]
20 False
[   0.          416.66666667 1000.        ]
21 False
[   0.   437.5 1000. ]
22 False
[   0.         

In [6]:
scene.frequency # Defaults to 3.5GHz

mat = ITURadioMaterial(name='concrete', itu_type='concrete', thickness=0.1, scattering_coefficient=0.0, xpd_coefficient=0.0)

# Set all objects to concrete for now
for obj in scene.objects.values():
    obj.radio_material = mat

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=1,
                          horizontal_spacing=1,
                          polarization="V",
                          pattern="iso")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")

# Create transmitter
tx = Transmitter(name="tx",
              position=[200,200,100],
              orientation=[0,0,0])
scene.add(tx)

# Create receivers
for rx_i in range(len(rx_points)):
    print(rx_points[rx_i])
    rx = Receiver(name=f"rx_{rx_i}",
            position=rx_points[rx_i],
            orientation=[0,0,0])
    scene.add(rx)

[ 83.33334351 312.5         25.        ]
[ 83.33334351 312.5         50.        ]
[ 83.33334351 312.5         75.        ]
[ 83.33334351 312.5        100.        ]
[ 83.33334351 312.5        125.        ]
[ 83.33334351 312.5        150.        ]
[ 83.33334351 312.5        175.        ]
[125.  312.5  25. ]
[125.  312.5  50. ]
[125.  312.5  75. ]
[125.  312.5 100. ]
[125.  312.5 125. ]
[125.  312.5 150. ]
[125.  312.5 175. ]
[145.83332825 312.49996948  25.        ]
[145.83332825 312.49996948  50.        ]
[145.83332825 312.49996948  75.        ]
[145.83332825 312.49996948 100.        ]
[145.83332825 312.49996948 125.        ]
[145.83332825 312.49996948 150.        ]
[145.83332825 312.49996948 175.        ]
[145.83334351 333.33331299  25.        ]
[145.83334351 333.33331299  50.        ]
[145.83334351 333.33331299  75.        ]
[145.83334351 333.33331299 100.        ]
[145.83334351 333.33331299 125.        ]
[145.83334351 333.33331299 150.        ]
[145.83334351 333.33331299 175.        ]

In [7]:
scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

In [8]:
p_solver  = PathSolver()
paths = p_solver(scene=scene,
                 max_depth=5,
                 los=False,
                 specular_reflection=True,
                 diffuse_reflection=False,
                 refraction=False,
                 synthetic_array=False,
                 seed=41)

In [9]:
scene.preview(paths=paths)


Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, matrixWorldNeedsUpda…